## 증권리포트 크롤링

- 매일경제증권센터 증권리포트 항목 크롤링
- 검색어, 검색할 maxPage 설정

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib import parse
import pandas as pd

#증권리포트 제목, 내용, url을 분리, 저장하는 함수
def get_report_inf(url):
    #headers = {'Accept-Encoding': 'deflate'}
    #res = requests.get(url=url, headers=headers)
    res = requests.get(url=url)
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')#, from_encoding="euc-kr"
    title = soup.find('p', {'class':'tit'}).text
    text = soup.find('div', {'id':'article_main'}).text
    report_data = {}
    report_data['title'] = title.replace("\r"," ").replace("\n"," ").replace("\t"," ").strip()  # 기사 제목
    report_data['text'] = text.replace("\n", " ").replace("\r", " ").replace("<br>", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
    report_data['url'] = url  # 기사 링크
    
    return report_data

In [2]:
#증권리포트 크롤링 data를 반환하는 함수

def get_report_list(page=1  # 뉴스 리스트 페이지
                      ):
    
    
    url='https://www.thebell.co.kr/free/content/article.asp?page={page}&svccode=00'
    
    res = requests.get(url.format(page=page))
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')
    #print(soup)
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출    
    table = soup.find('div', {'class':'listBox'})
    alinks = table.findAll('a')
    
    report_list = []
    
    for alink in alinks:
        alink['href'] = 'https://www.thebell.co.kr/free/content/' + alink['href']
        report_url = alink['href']
        report_list.append(get_report_inf(report_url))
    
    return report_list

In [3]:
#증권리포트 크롤링 실행
articles = []

for num in range(1, 21):
    articles.extend(get_report_list(page=num))

print("Total articles: {:,}".format(len(articles)))
#print("="*20, "Article sample", "="*20)
#print(articles[0])

Total articles: 200


In [4]:
df = pd.DataFrame(articles)
df[['title']].head(10)

title
0  대신자산신탁, 리츠 AMC 예비인가 신청 '속도'           설립인가 내년 ...
1  한미글로벌, 부동산개발 덕 '실적 껑충'           [Company Watc...
2  KTB증권, 우리은행 당산역지점 개발금융 주선           제이엔지파트너스 시...
3  대우조선해양건설, 인수 후 첫 성적표 '적자전환'           [건설리포트]올...
4  포스코건설, 내부거래 감소세 '지속'           [건설리포트]포스코 등 계열...
5  명동 티마크그랜드호텔 매각주관사 경쟁 '치열'           회계법인·자문사 7...
6  [아시아나항공 M&A]유력 후보 된 HDC현대산업개발, 재무여력 '탄탄'현금성 자산...
7  이지스운용, 신사동에 제2쌈지길 '가로골목' 오픈           총사업비 480...
8  서연, 서연전자 매각…재무개선 '청신호'           계열사 낮은 수익성에 고...
9  [해운업계 이합집산]2강 체제로 재편되는 인트라아시아 선사1위 고려해운 입지 넘보는...

## Text preprocessing

- 형태소 분류작업 (정규화, 어근화작업도 수행)

In [13]:
#from konlpy.tag import Okt
from konlpy.tag import Kkma
from collections import Counter
# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
#tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)
kkma = Kkma()

#키워드 추출 조건 설정
#input - 키워드로 포함하고자하는 품사 입력, 키워드로 포함하지 않을 금지단어 입력
#output - 기사갯수 100단위로 출력, 시data 첫번째 기사항목 출력
words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
stopwords = set([])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..
#stopwords.add(searchword)  #검색어 자체가 키워드에 포함되지 않게한다


for i, article in enumerate(articles):
        
    #words = tagger.pos(article['text'], norm=True, stem=True) #정규화, 어근화도 진행
    words = kkma.nouns(article['text'])
    tmp_words = []

    #for w, t in words:
    #    if t in tag_set and len(w) > 1 and w not in stopwords:
    #        tmp_words.append(w)
    vocab.update(words)
    words_list.append((words, article))
    


vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    # 기사 내 서로 다른 단어 개수가 5개 이상인 기사만 사용    
    if len(words) > 5:
        new_words_list.append((words, article))

In [14]:
#print(words)
#print(vocab)
#print(tagger.morphs(u'공부를 하면할수록 모르는게 많다는 것을 알게 됩니다.'))
print(new_words_list[0][0])

['편집', '편집자주', '자주', '1992', '1992년', '년', '우창', '우창록', '록', '변호사', '독립', '설립', '법무', '법무법인', '법인', '율촌', '역사', '국내', '대표적', '대형', '로펌', '성장', '초기', '조세', '공정', '공정거래', '거래', '분야', '전문', '기반', '송무', '기업자', '기업자문', '문', '영역', '확대', '존재', '존재감', '감', '각인', '2007', '2007년', '로펌으로', '최초', '베트남', '진출', '인도네시아', '러시아', '중앙', '중앙아시아', '아시아', '현지', '사무소', '해외', '법률', '자문', '자문시장', '시장', '벨', '면면', '세대', '이', '기사', '2019', '2019년', '08', '08월', '월', '30', '30일', '일', '10', '43', '유료', '유료페이지', '페이지', '표출', '2000', '2000년대', '년대', '화두', '대형화', '펌', '간의', '합병', '상위', '도약', '시도', '대신', '우수', '우수인재', '인재', '영입', '내부', '내부인력', '인력', '강화', '정공법', '선택', '특유', '교육', '교육시스템', '시스템', '도입', '측면', '작동', '수', '신', '신영수', '영수', '김규식', '등', '인사', '이진', '이진국', '국', '박재', '가세', '최근', '십', '년', '인수', '비약적', '직', '3', '3세대', '인수합병', '중심축', '역할', '전문영역', '구축', '김', '김건', '건', '김준형', '준형', '황', '황규상', '규', '상', '이수연', '4', '4세대', '두터워지', '평가', '포스트', '프', '프랜차이즈', '랜', '차', '이즈', '스타', '두각', '전문분야', '육성', '효과', '효과

## Build term-document matrix

In [7]:
#input - words_list, vocab
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd


print('기사 갯수 : ' ,len(words_list))
print('vocab 갯수 : ' ,len(vocab))
term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix)

기사 갯수 :  200
vocab 갯수 :  6403
Term document matrix shape: (200, 6403)
  (0, 6390)	0.02737635437725025
  (0, 6389)	0.026972524011058534
  (0, 6337)	0.06320878531517986
  (0, 6300)	0.06850595847619748
  (0, 6242)	0.02552942359460872
  (0, 6227)	0.030941890314320587
  (0, 6216)	0.07865868853453345
  (0, 6183)	0.018854326151613757
  (0, 6155)	0.09814214948497561
  (0, 6151)	0.033985498181928066
  (0, 6144)	0.01979233323286209
  (0, 6106)	0.03311344671235689
  (0, 6081)	0.06709692417363279
  (0, 6049)	0.023486194798145153
  (0, 6044)	0.030941890314320587
  (0, 5904)	0.032437053887963904
  (0, 5901)	0.046434554271771664
  (0, 5899)	0.030941890314320587
  (0, 5892)	0.008276871791280251
  (0, 5757)	0.011270305007225596
  (0, 5493)	0.03160439265758993
  (0, 5478)	0.02737635437725025
  (0, 5472)	0.03311344671235689
  (0, 5465)	0.046434554271771664
  (0, 5425)	0.033985498181928066
  :	:
  (199, 1490)	0.020569794202920107
  (199, 1461)	0.12044766397669349
  (199, 1335)	0.3036278676863367
  (199, 1

## Topic modeling

In [12]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition
from sklearn.decomposition import NMF
 
K = 20  # topic 개수지정
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (200, 20)
H shape: (20, 6403)


In [13]:
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:10]:
        print(vocab[index], end=' ')        
    print("\n", "="*50)

0th topic
상반기 수수료 수익 영업 일임 부문 감소 이익 손익 자산운용 
1th topic
펀드 자산운용 운용 설정 투자 헤지펀드 보수 출자 수익률 채권 
2th topic
바이오 브릿지 기술 퓨틱스 테라 스톤 마일 상장 개발 거인 
3th topic
부회장 뇌물 대법원 선고 전자 삼성 파기 환송 판결 재판 
4th topic
은행 금융 모형 지주 승인 부등 BNK 등급 DGB 경남 
5th topic
판매 DLS 상품 파생상품 금리 기초자산 DLF ELS 손실 불완전 
6th topic
모채 피코 발행 회사채 예측 조달 SK 금리 신용 수요 
7th topic
주가 코리아 CB 맥스 담보 에스 휠라 유니 자금 주주 
8th topic
에스엠 지배구조 개편 계열 크기 라이 적자 지주사 회사 열사 
9th topic
상장 IPO 증권 관사 기업 NIT 미래에셋 주관 대우 코스닥 
10th topic
프랭클린 운용 템플턴 합병 액티브 펀드 국내 자산운용 일임 투자신탁 
11th topic
SK 자문 인수 코웨이 웅진 변호사 네트웍 그룹 광장 율촌 
12th topic
사업 매출 건설 발전 베트남 GS 주택 부문 편의점 태양광 
13th topic
롯데 호텔 신라 채권 미즈 주류 제나 쇼핑 건설 이자보 
14th topic
해운 장금상선 부채 선사 컨테이너 법인 해운업 국적 분할 비율 
15th topic
KDB 생명 보험 중국 자본 CS 산업 제도 ICS 원매 
16th topic
금고 은행 기관 영업 국민은행 입찰 대출 카드 공무원 고객 
17th topic
창업 기술 버즈 스타트업 케이 기업 대표 처스 투자 라운드 
18th topic
GVA 부동산 자산운용 투자 파트너 펀드 담보 상업 제프리 대출 
19th topic
투자 해외 게임 중국 회수 컴투스 미국 벤처캐피탈 네트워크 KTB 


In [14]:
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    print("{}th topic".format(k))
    for index in W[:, k].argsort()[::-1][:5]:
        print(words_list[index][1]['title'])        
    print("\n", "="*80)

0th topic
VIP운용, 일임수수료 감소불구 순익 '껑충'           [헤지펀드 운용사 실적 분석]증시부진에 일임계약 성과보수 감소…고유계정 평가·처분 이익 증가
IBK증권, 구조화 선방…사라지는 IB 존재감           [하우스 분석]투자은행 부문, 손익 반토막…ECM·DCM 동반 부진
쿼드, 운용사 전환후 첫 적자...일임자산 축소 여파           [헤지펀드 운용사 실적 분석]펀드설정액·운용보수 증가불구 일임 수수료 급감…인력투자 확대 '압박'
이베스트증권, IB부문 성장 가시화…조직확대           [하우스 분석]수수료 수익 껑충…운용부문 쏠림 해소 숙제
유리치운용, 적자폭 확대…바닥 드러낸 일임보수           [헤지펀드 운용사 실적 분석]일임계약고 급감, 펀드 설정액 제자리 걸음

1th topic
한국벤처투자, '기준수익률 0%' 자펀드 비중 낮추나           국회예산정책처 "운용사 수익률 제고 노력 감소, 민간 자금 매칭 위축"
기업구조혁신펀드 순항, 출자재원 소진 임박           블라인드 6개·프로젝트 펀드 4개 성사
오라이언, 반기 첫 순익…부동산·메자닌펀드 견인           [헤지펀드 운용사 실적 분석]상반기말 펀드 설정액 2500억 돌파…실적 개선세 지속 전망
오라이언, 반기 첫 순익…부동산·메자닌펀드 견인           [헤지펀드 운용사 실적 분석]상반기말 펀드 설정액 2500억 돌파…실적 개선세 지속 전망
'고작 1bp'…타임폴리오 공모펀드 운용보수 의미는           이중수취 논란 방지…고객·판매사 확대 포석

2th topic
브릿지바이오, 기술성평가 '삼수' 성공…IPO 박차           평가기관 두곳에서 모두 'A'…내달 초 코스닥 예심 청구
브릿지바이오, L/O 450억 유입…흑자 바이오 된다           계약금 및 단기마일스톤 달성…조만간 두 차례 추가 수익 예상
'1.5조 기술이전' 브릿지바이오, IPO 스타트           내주 상장 예비심사 청

## Visualization

In [15]:
from sklearn.manifold import TSNE

print("W shape:", W.shape)
tsne = TSNE(n_components=2, init='pca', verbose=1) 
W2d = tsne.fit_transform(W)
topic_index = [v.argmax() for v in W]  # 각각의 문서가 어느 topic에 속해 있는지 (hard clustering)


W shape: (200, 20)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 200 samples in 0.000s...
[t-SNE] Computed neighbors for 200 samples in 0.000s...
[t-SNE] Computed conditional probabilities for sample 200 / 200
[t-SNE] Mean sigma: 0.139346
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.651028
[t-SNE] KL divergence after 950 iterations: 0.429205


In [16]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [17]:
tools_to_show = "hover,box_zoom,pan,save,reset,wheel_zoom"
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article['title'] for words, article in new_words_list],
    'topic': [str(i) for i in topic_index],
    'color': [Category20[K][i] for i in topic_index]
})



p.circle('x', 'y', source=source, legend='topic', color='color')

p.legend.location = 'top_left'
hover = p.select({'type': HoverTool})
hover.tooltips = [('Topic', '@topic'), ('id', '@id'), ('Article', '@document')]
hover.mode = 'mouse'

show(p)

In [18]:
#topic과 Keyword 분포, 빈도를 확인할 수 있는 visualization
#jupiter note에서 사용할때 그래프 확인후 interrupt the kernel로 종료 후 다음진행 필요
from sklearn.preprocessing import normalize

def y_to_doc_topic(y):
    n_topics = y.shape[1]
    base = 1 / n_topics
    doc_topic_prob = normalize(y, norm='l1')
    rowsum = doc_topic_prob.sum(axis=1)
    doc_topic_prob[np.where(rowsum == 0)[0]] = base
    return doc_topic_prob


In [19]:
def components_to_topic_term(components):
    n_terms = components.shape[1]
    base = 1 / n_terms
    topic_term_prob = normalize(components, norm='l1')
    rowsum = topic_term_prob.sum(axis=1)
    topic_term_prob[np.where(rowsum == 0)[0]] = base
    return topic_term_prob

In [20]:
doc_topic_prob = y_to_doc_topic(W)
topic_term_prob = components_to_topic_term(H)
doc_lengths = np.asarray(term_doc_matrix.sum(axis=1)).reshape(-1)
term_frequency = np.asarray(term_doc_matrix.sum(axis=0)).reshape(-1)

In [21]:
from pyLDAvis import prepare, show
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn


print(topic_term_prob.shape)
print(len(vocab))
prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    vocab,
    term_frequency,
    R = 10, #num of displayed terms,
    sort_topics = False
)

#display(prepared_data)
show(prepared_data)

(20, 6403)
6403


C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [02/Sep/2019 13:58:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2019 14:00:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2019 14:00:19] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2019 14:00:19] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2019 14:00:19] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...


## Simple search engine

In [22]:
query = '바이오'
query_index = word2id[query]  # 단어의 index

query_vec = term_doc_matrix[:, query_index]  # 각 문서에서 단어에 해당하는 값을 추출
nonzero_doc = query_vec.nonzero()[0]  # 0이 아닌 문서들만 사용
#print('논제로_독', nonzero_doc);
ranks = query_vec.toarray().flatten().argsort()[::-1]
#print('랭크', ranks);



for r in ranks[:10]:
    if r in nonzero_doc:
        print(new_words_list[r][1]['title'], new_words_list[r][1]['url'])


이지바이오, 마니커F&G 끝으로 계열사 IPO 쉼표           2016년부터 계열사 4곳 코스닥 입성…재무건전성 개선효과 https://www.thebell.co.kr/free/content/ArticleView.asp?key=201908290100053110003279&lcode=00&page=8&svccode=00
[바이오 CMO의 재발견]넥타, 제조이슈에 시총 급락…CMC 재조명④BMS와 4조 딜 체결하고 임상시료 품질 확보 못해…CDMO 활용이 성패 관리 '키' https://www.thebell.co.kr/free/content/ArticleView.asp?key=201908280100048590002997&lcode=00&page=4&svccode=00
브릿지바이오, 기술성평가 '삼수' 성공…IPO 박차           평가기관 두곳에서 모두 'A'…내달 초 코스닥 예심 청구 https://www.thebell.co.kr/free/content/ArticleView.asp?key=201908290100052800003262&lcode=00&page=14&svccode=00
'1.5조 기술이전' 브릿지바이오, IPO 스타트           내주 상장 예비심사 청구…기술성평가도 재신청 https://www.thebell.co.kr/free/content/ArticleView.asp?key=201908280100049160003029&lcode=00&page=17&svccode=00
인트론바이오, 300억 포기…미래 불확실성에 베팅           L/O 초기 마일스톤 포기…회사 측 "파마반트1 자금 문제 아냐, 4년뒤 상품화 주력" https://www.thebell.co.kr/free/content/ArticleView.asp?key=201908300100055710003441&lcode=00&page=5&svccode=00
브릿지바이오, L/O 450억 유입…흑자 바이오 된다           계약금 및 단기마일스톤 달성…조만간 두 차례